## Installing and importing dependencies

In [ ]:
!pip install transformers sacrebleu python-Levenshtein

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 51.8/51.8 kB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.0/104.0 kB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 162.6/162.6 kB 15.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 75.7 MB/s eta 0:00:00


In [ ]:
from google.colab import files
import json
import os
import random
from transformers import AutoModelForCausalLM, AutoTokenizer
import sacrebleu
import Levenshtein
import pandas as pd

## Dataset generation

For this experiment, I chose 37 random Python solutions from my past Leetcode exercises. These solutions will form the dataset for evaluating code completion models, specifically using [tiny_starcoder](https://huggingface.co/bigcode/tiny_starcoder_py), which is well-suited for Python code.

The following code is used to make dataset.

In [ ]:
def tokenize(code):
    lines = [line for line in code.splitlines(keepends=True) if line.replace("\n", "").replace("\t", "").strip() != ""]
    return lines

def split_code(lines):
    cursor = random.randint(1, len(lines)-3)
    middle_length = random.randint(1, 1)

    prefix = ""
    for val in lines[:cursor]:
        prefix += val
    suffix = ""
    for val in lines[cursor+middle_length:]:
        suffix += val
    middle = ""
    for val in lines[cursor]:
        middle += val

    return prefix, suffix, middle

def generate(directory_path):
    data = []
    for filename in os.listdir(directory_path):
        file_path = os.path.join(directory_path, filename)
        with open(file_path, 'r') as file:
            code = file.read()
            lines = tokenize(code)
            prefix, suffix, middle = split_code(lines)
            data.append({
                "input": f"<fim_prefix>{prefix}<fim_suffix>{suffix}<fim_middle>",
                "output": middle
                })

    with open("data.json", 'w') as json_file:
        json.dump(data, json_file, indent=4)

generate("data")

**`tokenize(code)`**: Splits the input code into a list of non-empty lines, keeping line endings. It filters out any lines that are just whitespace.

**`split_code(lines)`**: Randomly selects a line (`cursor`) to split the tokenized lines into three parts:
   - **Prefix**: Lines before the cursor.
   - **Middle**: The line at the cursor.
   - **Suffix**: Lines after the cursor.

**`generate(directory_path)`**: Iterates through files in the specified directory and generates a dataset:
   - Reads each file, tokenizes the code, and splits it using `split_code`.
   - Appends the formatted data (input and output) to a list.
   - Saves the dataset as a JSON file named `data.json`.

The final output format for each entry is structured for a code completion model, with special tokens indicating the different sections.

## Uploading the dataset used for evaluation
You can find the dataset generated from my code examples, which was created using the code provided above, [here](https://github.com/Olivera2708/Code-completion/blob/main/data.json). This dataset will be used for evaluating code completion models.

In [ ]:
uploaded = files.upload()

Saving data.json to data.json


## Generation of Missing Code
This Python code is designed to generate the missing middle section of code snippets and evaluate it using multiple metrics.

In [ ]:
def token_match(pred, target):
    pred_tokens = set(pred.split())
    target_tokens = set(target.split())
    return len(pred_tokens & target_tokens) / len(target_tokens) if target_tokens else 0

def is_runnable(code):
    try:
        exec(code)
        return True
    except Exception as e:
        return False


checkpoint = "bigcode/tiny_starcoder_py"
device = "cuda"

tokenizer = AutoTokenizer.from_pretrained(checkpoint)
model = AutoModelForCausalLM.from_pretrained(checkpoint).to(device)

params = {
    'max_new_tokens': 30
}

with open("data.json", 'r') as file:
    data = json.load(file)

correct_full = 0
results = []

for i, element in enumerate(data):
    input = element["input"]
    output = element["output"]

    inputs = tokenizer.encode(input, return_tensors="pt").to(device)
    outputs = model.generate(inputs, pad_token_id=tokenizer.eos_token_id, **params)
    generated_output = tokenizer.decode(outputs[0])

    start = generated_output.find("<fim_middle>") + len("<fim_middle>")
    end = generated_output.find("<|endoftext|>")

    generated = generated_output[start:end]
    code = input.replace("<fim_prefix>", "").replace("<fim_suffix>", generated).replace("<fim_middle>", "")

    results.append({
        "correct": output,
        "generated": generated,
        "exact match": output == generated,
        "chrf": sacrebleu.corpus_chrf([generated], [[output]]).score,
        "levenshtein": Levenshtein.distance(generated, output),
        "token match": token_match(generated, output),
        "runnable": is_runnable(code)
    })

    if output == generated_output[start:end]:
        correct_full += 1

with open("results_generated.json", 'w') as file:
        json.dump(results, file, indent=4)

print(f"Exact matches -> {correct_full} out of {len(data)}")

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/677 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/777k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/442k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.06M [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/1.03k [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/657M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/111 [00:00<?, ?B/s]

The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Exact matches -> 8 out of 37


**Load Dataset**: It reads a JSON file (`data.json`) containing input-output pairs for evaluation.

**Generate Missing Code**:
   - For each input, it encodes the input text and generates the output using the model.
   - The generated text is processed to extract the missing middle code segment.
   - It compares the generated output with the expected output and counts correct matches.

**Save Results**: The results, including the generated code, correct code are saved to `results_generated.json`.


## Manual evaluation of the results
This code is used as an easier way to check what are the differences.

I reviewed the generated code to identify the key differences. Based on this, I assigned five labels to the outputs:

- **Correct**: The generated code matches the expected output exactly.
- **Partially Correct**: The generated code is mostly accurate but may have minor errors, missing elements, or contain some extra code that doesn't belong.
- **Incorrect**: The generated code deviates significantly from the correct solution and contains logical or structural issues.
- **Code that runs**: The generated code may not be perfect, but it is functional and will run without errors.
- **Code that does not run**: The generated code is flawed and will not execute properly, containing syntax errors or incomplete logic.

In [ ]:
with open("data.json", 'r') as file:
    data = json.load(file)

with open("results.json", 'r') as file:
    results = json.load(file)

for i, example in enumerate(data):
    input = example["input"].replace("<fim_prefix>", "").replace("<fim_middle>", "").replace("<fim_suffix>", "MISSING\n")
    print(f"----- Example {i+1} -----\n")
    print(input)
    print(f"--- Generated ---\n")
    print(results[i]["generated"])
    print(f"---  Correct  ---\n")
    print(results[i]["correct"])
    print()

----- Example 1 -----

class Solution:
    def maxProfit(self, prices):
        max = 0
MISSING
        for i in range(1, len(prices)):
            if start < prices[i]: 
                max += prices[i] - start
            start = prices[i]
        return max
--- Generated ---

        for i in range(1, len(prices)):
            if prices[i] > max:
                max = prices[i]

---  Correct  ---

        start = prices[0]


----- Example 2 -----

class Solution:
	def isPalindrome(self, s):
		s = s.lower()
		start = 0
		end = len(s) - 1
		while start < end:
			if not s[start].isalnum():
				start += 1
			elif not s[end].isalnum():
				end -= 1
			elif s[start] != s[end]:
MISSING
			else:
				start += 1
				end -= 1
		return True
--- Generated ---

				return False

---  Correct  ---

				return False


----- Example 3 -----

class Solution():
	def longestCommonPrefix(self, strs):
		i = 0
		prefix = ""
		if not strs:
			return prefix
		while True:
			if i == len(strs[0]):
				return 

### Correct

In these examples, the generated code correctly matches the intended solution:

- *Example 2*, *Example 10*, *Example 15*, *Example 19*, *Example 22*, *Example 23*, *Example 31*, *Example 32*.

### Partially Correct
The generated code often includes correct portions but is accompanied by extra, irrelevant code that should not be part of the solution.

- *Example 4*: The first line of the generated code is correct.
- *Example 6*: The generated code contains an incomplete `if` statement, missing key components.
- *Example 8*: A partially correct `while` loop was generated, though the part after the `or` is missing. Additionally, extra code was included, mainly copied from the suffix.
- *Example 17*: The first line of the generated code is correct but is indented by one tab less than it should be.
- *Example 21*: Unnecessary comments and an `if` statement were generated; the `if` is almost correct.
- *Example 25*: The first line of the generated code is correct.
- *Example 26*: The first line of the generated code is correct.
- *Example 30*: The first line of the generated code is correct, but the remainder contains code copied from the suffix.
- *Example 34*: The first line of the generated code is correct.
- *Example 35*: The variable in the `for` loop is incorrect—`m` was generated instead of `n`.
- *Example 37*: The first line is nearly correct, but the comparison sign is incorrect.

### Incorrect

The following examples include significant errors, with generated code either missing crucial elements or not aligning with the intended solution.

- *Example 1*: The `start` variable is not initialized. The generated code attempts to find the maximum value in the array, which is unrelated to the function's goal.
- *Example 3*: The code is incomplete. It seems the model ignored the suffix, as parts of the generated code also appear in the suffix. This might be due to the presence of two `if` loops with the same content.
- *Example 5*: The generated code compares incorrect values and omits some parts.
- *Example 7*: No code was generated, only a tab (whitespace).
- *Example 9*: The code is partially copied from the suffix and does not make sense.
- *Example 11*: The generated code is nonsensical.
- *Example 12*: The generated code is nonsensical.
- *Example 13*: The code does not make sense, with some parts copied from the prefix.
- *Example 14*: Part of the code is copied from the suffix.
- *Example 16*: Part of the code is copied from the suffix.
- *Example 18*: Part of the code is copied from the suffix.
- *Example 20*: The code is nonsensical, particularly in how it compares values.
- *Example 24*: The `dict` variable is not initialized, only a tab (whitespace) is present.
- *Example 25*: The code doesn't make sense, and it's unclear what should have been generated.
- *Example 28*: No code was generated, only comments.
- *Example 29*: The generated code does not make sense.
- *Example 33*: The generated code does not make sense.
- *Example 36*: The `i` variable is not initialized.

### Code That Runs

The following examples generated code that is executable, even if not always entirely correct:

- *Example 2*, *Example 10*, *Example 15*, *Example 19*, *Example 22*, *Example 23*, *Example 28*, *Example 31*, *Example 32*, *Example 33*, *Example 35*.

### Code That Does Not Run

These examples contain errors that prevent the code from executing:

- *Example 1*: Fails due to uninitialized `start` variable and incorrect indentation.
- *Example 3*: Unfinished code with incorrect indentation.
- *Example 4*: Random code.
- *Example 5*: The `current` variable is never assigned.
- *Example 6*: Incorrect indentation.
- *Example 7*: Missing `sett` variable.
- *Example 8*: Unfinished code with incorrect indentation.
- *Example 9*: Incorrect indentation.
- *Example 11*: Unfinished code.
- *Example 12*: Unfinished code.
- *Example 13*: Incorrect indentation.
- *Example 14*: Missong `if` statement, just `else` present.
- *Example 16*: Unfinished code.
- *Example 17*: Unfinished code.
- *Example 18*: Unfinished code, includes random comments and the start of a new class.
- *Example 20*: Incorrect indentation.
- *Example 21*: Error due to accessing an out-of-bounds index.
- *Example 24*: Missing `dict` variable.
- *Example 25*: Incorrect indentation.
- *Example 27*: Incorrect indentation.
- *Example 26*: `if` before `elif` is not allowed.
- *Example 29*: Unfinished code, includes random comments and the start of a new class.
- *Example 30*: Unfinished code.
- *Example 34*: Incorrect indentation.
- *Example 36*: Missing `dict` variable.
- *Example 37*: Incorrect indentation.

## Metrics

### Exact Match Evaluation

The exact match metric assesses whether the predicted code exactly matches the correct completion, character for character.

This metric aligns with the "correct" classification used during the manual labeling process.

### CHRF Metric

CHRF (Character n-gram F-score) is a character-level evaluation metric frequently used in natural language processing (NLP). It measures the similarity between the predicted output and the ground truth by analyzing character n-grams, which are contiguous sequences of characters of a specified length.

A value of 100.0 indicates an exact match, while a value of 0.0 signifies that the model generated only whitespace without any code.

### Levenshtein Distance

Levenshtein distance quantifies the number of edits—insertions, deletions, and substitutions—needed to change the predicted code into the correct code. This metric reveals how closely two strings align by counting the minimum edits required for one to become the other.

A distance of 0 indicates an exact match.

A large distance (in this example) suggests that the model has generated more code than necessary, reflecting significant differences from the expected output.

### Token Match

The Token Match metric evaluates the overlap between tokens in the predicted code and the expected code. It quantifies how many tokens from the predicted output are present in the ground truth, offering insight into the alignment between the two.

This metric is similar to the "partially correct" classification I used, yielding a value of 1.0 when the first line matches the expected output, regardless of the remaining code. However, there are other examples I labeled as partially correct, where the logic is similar, but this metric does not account for the logic of the code.

### Runnable Match

The Runnable Match metric assesses whether the predicted code is syntactically correct and can be executed without errors. This evaluation goes beyond simple text comparison, emphasizing the code's capability to run successfully in a programming environment.

This metric is like my "runnable" label, but it's important to note that some code may appear runnable even when it isn't. For instance, if a function is defined but not called, the metric will indicate that the code is runnable, despite missing initializations of some variables inside the function.

## Viewing the results

You can find the dataset generated from my code and manually written labels combined [here](https://github.com/Olivera2708/Code-completion/blob/main/complete_results.json).

In [ ]:
uploaded = files.upload()

Saving complete_results.json to complete_results.json


In [ ]:
with open('complete_results.json', 'r') as file:
    generated = json.load(file)

pd.DataFrame(generated)

,correct,generated,exact match,chrf,levenshtein,token match,runnable,label,runnable_manual
0,start = prices[0]\n,"for i in range(1, len(prices)):\n ...",False,29.013859,82,0.333333,True,incorrect,False
1,\t\t\t\treturn False\n,\t\t\t\treturn False\n,True,100.000000,0,1.000000,True,correrct,True
2,\t\t\t\t\treturn prefix\n,\t\t\t\tprefix += p\n\t\t\ti += 1\n\t\t\tprefi...,False,36.414045,61,0.500000,False,incorrect,False
3,}\n,}\n result = []\n for i ...,False,7.042254,134,1.000000,False,partial,False
4,"current = max(prev1, prev2 + num)\n",if num > prev1:\n p...,False,25.685825,59,0.166667,True,incorrect,False
5,while len(to_visit) > 0 or len(next_vi...,while to_visit:\n node = to...,False,39.952425,90,0.142857,False,partial,False
6,\t\tsett = set()\n,\t,False,0.000000,14,0.000000,False,incorrect,False
7,\t\t\tif (s[i] in map1 and map1[s[i]] != t[i])...,\t\t\tif s[i] in map1:\n\t\t\t\tmap1[s[i]] = m...,False,34.607912,46,0.250000,False,partial,False
8,head.next = None\n,fast = head\n previous = head\n...,False,22.529454,59,0.333333,True,incorrect,False
9,current = self.root\n,current = self.root\n,True,100.000000,0,1.000000,True,correrct,True


### My Thoughts

Reflecting on the results from this experiment, several insights stand out:

1. **Evaluation Metrics**: The use of multiple metrics (Exact Match, CHRF, Levenshtein Distance, Token Match, and Runnable Match) provides a comprehensive view of the model's performance. However, these metrics cannot fully replace manual labeling due to the complexity of code logic. For instance, Runnable Match only checks for syntactic correctness—it doesn't account for whether the generated code solves the problem as intended. Manual labeling, by contrast, allows for a more nuanced evaluation, capturing logical correctness and contextual appropriateness, which purely automatic metrics often miss.

2. **Manual Labeling**: The manual labeling process provided a deeper understanding of the mistakes made by the model. By reviewing the generated code closely, I could pinpoint recurring errors, such as incorrect indentation, over-generating code, or producing empty lines. While it was tempting to break these down into smaller categories, I ultimately decided that the current labels—Correct, Partially Correct, Incorrect, Runnable, and Not Runnable—capture the essence of the errors without over-complicating the analysis.

3. **Common Errors**: Many of the "Incorrect" examples displayed recurring patterns, such as uninitialized variables, improper syntax, or missing logic. These repeated mistakes suggest areas where the model may require further refinement, such as handling variable initialization or indentation more effectively. Another common issue was the generation of excessive code or empty lines, often leading to unnecessary complexity in the output. A notable trend was the generation of comments in the middle of the code, which likely stems from the dataset that the model was trained on.